In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image
from scipy import ndimage
from sklearn.model_selection import train_test_split

# ! pip install tensorflow
import tensorflow as tf
from tensorflow.keras import layers, models

np.random.seed(23)
plt.rcParams['figure.figsize'] = (4, 4)
plt.rcParams['figure.dpi'] = 150
sns.set()

### TO DO 

We really need to create functions that take in an image array and spit out a single feature value that could be added to a vector (I also want to talk to the professor about whether this is the most effective way to do this).

In [18]:
def extract_log_features(image_array, sigma=0.7, scalar=True):
    """
    Extract Laplacian of Gaussian (LoG) features from an image array.
    
    Parameters:
        image_array (numpy.ndarray): The input image array.
        sigma (float): The sigma value for the Gaussian filter. Controls the amount of smoothing.
        
    Returns:
        numpy.ndarray: The LoG filtered image as a feature vector.
    """
    # Apply the Laplacian of Gaussian filter
    log_image = ndimage.gaussian_laplace(image_array, sigma=sigma)

    if scalar:
        
        # OPTION 1
        
        # feature scalar: the sum of absolute values in the LoG image (a simple measure of edginess)
        feature_scalar = np.sum(np.abs(log_image))

        return feature_scalar

    else:
        # OPTION 2
        
        # feature vector: flatten the LoG image to use as a feature vector directly
        feature_vector = log_image.flatten()

        return feature_vector

In essence, each image needs to be represented as a feature vector. In the D200 example, the photos of the clothing items were in grayscale and then normalized and then reshaped using `train_images_vectors = np.reshape(train_images, (len(train_images), -1))` where train images is an array of n 28 x 28 training images (i.e. train_images.shape = (n, 28, 28)). After each image is reshaped, the vector is 1 x 784 (i.e. 28 x 28).

We need a dataframe at the end of the day, where each row represents an image and its features. I want to talk more about which features we are actually going to leverage and use as a team, but for right now, I will build the plumbing to be able to run a classification once we actually have the features using the 'full image' technique employed in D200. In our case the images are still 200 x 200.

In [19]:
def parse_data(folder_path):
    image_vectors = []  # image data
    labels = []  # labels
    ids = []  # unique IDs

    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            parts = filename.split('_')
            fabType = parts[0]
            id1 = parts[1]
            id2 = parts[3].split('.')[0]  # Remove .png extension
            
            unique_id = id1 + id2

            img = Image.open(os.path.join(folder_path, filename)).convert('L')
            img_array = np.array(img)
            
            # normalize the image vector to be between 0 and 1
            img_vector_normalized = img_array.flatten() / 255.0

            log_scalar = extract_log_features(img_array)
            new_vector_with_scalar = np.append(img_vector_normalized, log_scalar)

            #### TO DO - MAJOR ####
            #### ADD FEATURES IN HERE #####
            
            image_vectors.append(img_vector_normalized)
            labels.append(fabType)
            ids.append(unique_id)

    X = np.array(image_vectors)
    Y = np.array(labels)
    unique_ids = np.array(ids)
    return X, Y, unique_ids   

Note - the cell below takes ~25 seconds to run on my mac. It may take a bit longer depending on your processing power and memory. If this becomes an issue, we can save the dataframe it creates as a pickle file, which could be saved to sub-samples and you two would be able to use without issue i.e. my computer processes and adds features and then saves it as a compressed python binary to allow for easy access that avoids double processing later. Just let me know if we need to do that!

In [20]:
folder_path = './Subsamples/train'
X, Y, unique_ids = parse_data(folder_path)

df = pd.DataFrame(X)
df['category'] = pd.Categorical(Y)
df['label'], _ = pd.factorize(df['category'])
df['uid'] = unique_ids
df.head()

,0,1,2,3,4,5,6,7,8,9,...,39993,39994,39995,39996,39997,39998,39999,category,label,uid
0,0.062745,0.058824,0.066667,0.062745,0.062745,0.066667,0.082353,0.086275,0.101961,0.101961,...,0.066667,0.058824,0.054902,0.054902,0.058824,0.062745,0.062745,Blended,0,8821c
1,0.266667,0.258824,0.270588,0.301961,0.313725,0.352941,0.439216,0.478431,0.572549,0.607843,...,0.200000,0.333333,0.749020,0.803922,0.788235,0.760784,0.717647,Denim,1,1503c
2,0.235294,0.247059,0.274510,0.298039,0.309804,0.305882,0.282353,0.274510,0.270588,0.258824,...,0.168627,0.133333,0.125490,0.133333,0.160784,0.192157,0.211765,Polyester,2,16132c
3,0.380392,0.364706,0.356863,0.345098,0.341176,0.345098,0.388235,0.435294,0.450980,0.423529,...,0.400000,0.415686,0.450980,0.482353,0.482353,0.505882,0.545098,Blended,0,3621d
4,0.321569,0.317647,0.305882,0.298039,0.294118,0.301961,0.309804,0.313725,0.325490,0.333333,...,0.376471,0.298039,0.247059,0.278431,0.345098,0.333333,0.309804,Cotton,3,2333a


With this dataframe, which is in the same format as the data shown in the MNIST D200 homework, it should be easy to set up t-SNE and PCA. HOWEVER - at this point we don't have any true features, as our existing features need to be converted to columns in this dataframe. Below is skeleton code that can be filled out to run and add these features

## TO DO - INSERT PCA AND OTHER EDA

In [26]:
# training/validation split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# simple cnn model
# def create_cnn_model(input_shape, num_classes):
#     model = models.Sequential([
#         # convolutional layer with ReLU activation and Max Pooling
#         layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
#         layers.MaxPooling2D((2, 2)),
        
#         # convolutional layer 2
#         layers.Conv2D(64, (3, 3), activation='relu'),
#         layers.MaxPooling2D((2, 2)),
        
#         # flatten the output and add dense layers for classification
#         layers.Flatten(),
#         layers.Dense(64, activation='relu'),
#         layers.Dense(num_classes, activation='softmax')
#     ])
    
#     return model

# input_shape = (64, 64, 3)  # input shape (height, width, channels)
# num_classes = 5

# model = create_cnn_model(input_shape, num_classes)

# # compile the model
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])


# # train_images, train_labels = ... 
# # val_images, val_labels = ... 

# train model
# history = model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))


In [ ]:
## TO DO - ADD SVM CODE